In [1]:
import os
import sys

os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
    
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
        .builder
        .config(conf=conf)
        .appName('Lab01')
        .getOrCreate())

In [3]:
spark

In [4]:
film_id = '328'

In [5]:
rdd_rate = sc.textFile("/labs/laba01/ml-100k/u.data")

In [6]:
rdd_rate = rdd_rate.map(lambda x: x.split("\t"))

In [7]:
rdd_rate.take(5)

# user id | item id | rating | timestamp

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [8]:
rdd_rate.count()

100000

In [9]:
# 1. Статистика для выбранного фильма

# оставляем строки только для нужного фильма
rdd_rate_film = rdd_rate.filter(lambda x: x[1] == film_id)

# оставляем только значение оценки и 1 в кортеже для последующей агрегации
rdd_rate_film = rdd_rate_film.map(lambda x: (x[2],1))

# суммируем 1 из кортежей по каждой оценке
hist_film = rdd_rate_film.map(lambda x: (x[0], 1)). \
    reduceByKey(lambda x, y: x + y). \
    collect()

# сортируем получившуюся статистику по оценкам от 1 до 5
hist_film = sorted(hist_film, key=lambda x: x[0])
hist_film

[('1', 12), ('2', 40), ('3', 94), ('4', 109), ('5', 40)]

In [10]:
# оставляем только кол-во оценок
hist_film = [x[1] for x in hist_film]
hist_film

[12, 40, 94, 109, 40]

In [11]:
# 2. Статистика по всем фильмам

# считаем аналогично

rdd_rate_all = rdd_rate.map(lambda x: (x[2],1))

hist_all = rdd_rate_all.map(lambda x: (x[0], 1)). \
    reduceByKey(lambda x, y: x + y). \
    collect()

hist_all = sorted(hist_all, key=lambda x: x[0])
hist_all

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

In [12]:
hist_all = [x[1] for x in hist_all]
hist_all

[6110, 11370, 27145, 34174, 21201]

In [13]:
result = {"hist_film":hist_film, "hist_all":hist_all}

In [14]:
result

{'hist_film': [12, 40, 94, 109, 40],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [15]:
with open("lab01.json", "w") as fp:
    json.dump(result , fp) 

In [16]:
spark.stop()